# Proyecto - Asesor legal para consultar historia de demandas.
## Generación de embeddings 

Para esta parte del proyecto se seguirá el siguiente esquema en código 

```mermaid
    graph LR 
        A[ 1 <br> Cargar el dataset limpio]-->B[2 <br> Embeddings con SentenceTransformers]
        B-->C[3 <br> Vector Store con Chroma ]
```

### 1. Cargar el dataset limpio 

In [1]:
import pandas as pd

df_full = pd.read_csv("C:/Users/nat27/Desktop/Desktop/Proyectos/CienciaDatos/legal-consulting-poc/data/clean/sentencias_limpio_full.csv")


# Nos enfocamos a los casos marcados como redes sociales 
df_rag = df_full[df_full["is_redes_sociales"]].copy()

# Reasegurar texto base para embeddings (si no existe, lo reconstruimos rápido)
def build_document(row):
    partes = []
    if pd.notna(row.get("providencia")):   partes.append(f"Providencia: {row['providencia']}")
    if pd.notna(row.get("tema_subtema")):  partes.append(f"Tema/Subtema: {row['tema_subtema']}")
    if pd.notna(row.get("sintesis")):      partes.append(f"Sintesis: {row['sintesis']}")
    if pd.notna(row.get("resuelve")):      partes.append(f"Resuelve: {row['resuelve']}")
    fecha = row["fecha_sentencia"] if "fecha_sentencia" in row and pd.notna(row["fecha_sentencia"]) else "Desconocida"
    partes.append(f"Fecha sentencia: {fecha}")
    return "\n".join([p for p in partes if p])

if "documento" not in df_rag.columns:
    df_rag["documento"] = df_rag.apply(build_document, axis=1)

# Asegura ids como str
df_rag["doc_id"] = df_rag["doc_id"].astype(str)
len(df_rag), df_rag.columns.tolist()[:10]


(180,
 ['id_raw',
  'relevancia',
  'providencia',
  'tipo',
  'fecha_sentencia',
  'tema_subtema',
  'resuelve',
  'sintesis',
  'doc_id',
  'anio'])

### 2. Embeddings con sentence transformes 

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model_name = "all-MiniLM-L6-v2"  
embedder = SentenceTransformer(embedding_model_name)

# Calculamos embeddings por lotes 
texts = df_rag["documento"].tolist()
embeddings = embedder.encode(texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True)

# Guardamos en RAM 
embeddings.shape

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

(180, 384)